In [1]:
import os
import time
from dataclasses import dataclass
from pathlib import Path

import tensorflow as tf

# Optional during stabilization; disable for performance once stable
# tf.config.run_functions_eagerly(True)
# Optional: tf.data debug mode (slow) - only when debugging input pipelines
# tf.data.experimental.enable_debug_mode()

print("TensorFlow:", tf.__version__, "| Eager:", tf.executing_eagerly())

TensorFlow: 2.20.0 | Eager: True


In [2]:
%pwd

'f:\\Git Projects\\Venkey-Chicken-Disease-Classification\\research'

In [3]:
import os
os.chdir("../")  # from research/ to project root
%pwd

'f:\\Git Projects\\Venkey-Chicken-Disease-Classification'

In [ ]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories

In [5]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    learning_rate: float

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        cfg = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(cfg.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(cfg.tensorboard_root_log_dir)])
        return PrepareCallbacksConfig(
            root_dir=Path(cfg.root_dir),
            tensorboard_root_log_dir=Path(cfg.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(cfg.checkpoint_model_filepath)
        )

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([Path(training.root_dir)])

        return TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=int(params.EPOCHS),
            params_batch_size=int(params.BATCH_SIZE),
            params_is_augmentation=bool(params.AUGMENTATION),
            params_image_size=list(params.IMAGE_SIZE),
            learning_rate=float(params.LEARNING_RATE),
        )

In [7]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True,
            monitor="val_loss",
            mode="min",
            verbose=1
        )

    @property
    def _create_early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True,
            verbose=1
        )

    @property
    def _create_reduce_lr(self):
        return tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=2,
            min_lr=1e-6,
            verbose=1
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_early_stopping,
            self._create_reduce_lr
        ]


In [8]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None
        self.train_generator = None
        self.valid_generator = None
        self.steps_per_epoch = None
        self.validation_steps = None

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagen = valid_datagen

        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        print("Eager at fit:", tf.executing_eagerly())

        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Do not pass workers/use_multiprocessing/max_queue_size to avoid warnings with this adapter
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list,
            verbose=1
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [9]:
try:
    cfg_mgr = ConfigurationManager()

    # Prepare callbacks
    cb_cfg = cfg_mgr.get_prepare_callback_config()
    cb_prep = PrepareCallback(config=cb_cfg)
    callback_list = cb_prep.get_tb_ckpt_callbacks()

    # Training pipeline
    train_cfg = cfg_mgr.get_training_config()
    trainer = Training(config=train_cfg)
    trainer.get_base_model()
    trainer.train_valid_generator()
    trainer.train(callback_list=callback_list)

except Exception as e:
    raise e


[2025-08-16 10:42:26,025: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-16 10:42:26,028: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-16 10:42:26,029: INFO: common: created directory at: artifacts]
[2025-08-16 10:42:26,030: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-08-16 10:42:26,032: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-08-16 10:42:26,033: INFO: common: created directory at: artifacts\training]
Found 1155 images belonging to 7 classes.
Found 4636 images belonging to 7 classes.
Eager at fit: True


f:\Git Projects\Venkey-Chicken-Disease-Classification\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


289/289 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6314 - loss: 7.1226

f:\Git Projects\Venkey-Chicken-Disease-Classification\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_loss improved from None to 4.21015, saving model to artifacts\prepare_callbacks\checkpoint_dir\model.keras
289/289 ━━━━━━━━━━━━━━━━━━━━ 373s 1s/step - accuracy: 0.6851 - loss: 6.0284 - val_accuracy: 0.7917 - val_loss: 4.2101 - learning_rate: 0.0100
Restoring model weights from the end of the best epoch: 1.
